In [132]:
from llama_parse import LlamaParse
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

QDRANT_API = os.getenv("QDRANT_API")
QDRANT_URL = os.getenv("QDRANT_URL")

LLAMA_PARSE_API = os.getenv("LLAMA_PARSE_API")

VOYAGE_API = os.getenv("VOYAGE_API")

In [133]:
# file_path ='data/Kazakhstan_tarihi_7_atamura_sample.pdf'

DISCIPLINE = "Қазақстан тарихы"
GRADE = "6 сынып"
PUBLISHER = "Атамұра"

collection_name = "JauapAI"

In [198]:
import uuid
import re
from qdrant_client import QdrantClient, models
from langchain_text_splitters import MarkdownHeaderTextSplitter


from FlagEmbedding import BGEM3FlagModel
from langchain_voyageai import VoyageAIEmbeddings



# Initialize Voyage Large
dense_model = VoyageAIEmbeddings(
    voyage_api_key=VOYAGE_API, 
    model="voyage-3-large",
    output_dimension=1024
)
sparse_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API)


# 1. Создание коллекции (Hybrid)
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            "voyage-dense": models.VectorParams(
                size=1024, # Размер для Gemini
                distance=models.Distance.COSINE
            )
        },
        sparse_vectors_config={
            "bge-sparse": models.SparseVectorParams(
                index=models.SparseIndexParams(on_disk=True)
            )
        }
    )
    print(f"Collection '{collection_name}' created.")
else:
    print(f"Collection '{collection_name}' already exists.")


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 105649.97it/s]


Collection 'JauapAI' created.


In [38]:
full_markdown_list = []
full_markdown = ""
image_vault = {}


In [ ]:



# 2. Парсинг и семантическое разделение
# parser = LlamaParse(
#     api_key=LLAMA_PARSE_API,
#     parse_mode="parse_page_with_lvm",
#     model="gemini-2.5-ro",
#     vendor_multimodal_api_key=GEMINI_API_KEY,
#     result_type="markdown",
#     high_res_ocr=True,
#     adaptive_long_table=True,  # Adaptive long table. LlamaParse will try to detect long table and adapt the output
#     outlined_table_extraction=True,  # Whether to try to extract outlined tables
# )

parser = LlamaParse(
    api_key=LLAMA_PARSE_API,
    parse_mode="parse_page_with_lvm",
    model="openai-gpt-4-1-mini",
    vendor_multimodal_api_key=OPENAI_API_KEY,
    result_type="markdown",
    high_res_ocr=True,
    adaptive_long_table=True,  # Adaptive long table. LlamaParse will try to detect long table and adapt the output
    outlined_table_extraction=True,  # Whether to try to extract outlined tables
    num_workers=1,
    max_timeout=300,
    job_timeout_in_seconds=300,
    system_prompt_append="Extract the text exactly as it appears"
)


In [37]:
import os
import uuid
import time
from PyPDF2 import PdfReader, PdfWriter
from llama_parse import LlamaParse

# --- CONFIGURATION ---
file_path = "data/Kazakh_history_6.pdf"
CHUNKS_DIR = "data/temp_chunks"
BATCH_SIZE = 2  # Number of pages per batch

if not os.path.exists(CHUNKS_DIR):
    os.makedirs(CHUNKS_DIR)

def split_pdf(path, batch_size):
    reader = PdfReader(path)
    total_pages = reader.pages.__len__()
    chunk_paths = []
    
    for i in range(0, total_pages, batch_size):
        writer = PdfWriter()
        end_page = min(i + batch_size, total_pages)
        for page_num in range(i, end_page):
            writer.add_page(reader.pages[page_num])
        
        chunk_filename = f"{CHUNKS_DIR}/chunk_{i}_{end_page}.pdf"
        with open(chunk_filename, "wb") as f:
            writer.write(f)
        chunk_paths.append(chunk_filename)
    
    return chunk_paths

# --- MAIN PARSING LOGIC ---

chunk_files = split_pdf(file_path, BATCH_SIZE)


In [39]:

print(f"Total chunks to process: {len(chunk_files)}")

for idx, chunk_file in enumerate(chunk_files):
    stop = True
    retry_count = 0
    
    while stop and retry_count < 10:
        try:
            print(f"[{idx+1}/{len(chunk_files)}] | {chunk_file}")

            if not os.path.exists(chunk_file):
                raise FileNotFoundError(f"Chunk file not found: {chunk_file}")
            
            # Using get_json_result for the specific chunk
            json_results = parser.get_json_result(chunk_file)
            pages = json_results[0].get("pages", [])

            if not pages:
                raise ValueError(f"No pages returned from parser {json_results[0]["error"]}")
            
            chunk_markdown = ""
            
            # Process results for this chunk
            for i, page in enumerate(pages, start=1):
                full_markdown += f"START OF PAGE: {int(chunk_file.split("_")[-2])+i}\n"
                full_markdown += page.get("md", "") + "\n"
                full_markdown += f"\nEND OF PAGE: {int(chunk_file.split("_")[-2])+i}\n"

                chunk_markdown += f"START OF PAGE: {int(chunk_file.split("_")[-2])+i}\n"
                chunk_markdown += page.get("md", "") + "\n"
                chunk_markdown += f"\nEND OF PAGE: {int(chunk_file.split("_")[-2])+i}\n"
                
                for img in page.get("images", []):
                    # We add chunk index to image name to avoid collisions
                    img_name = f"chunk_{idx}_{img.get('name')}"
                    print()
                    print(f"Processing image: {img_name}")
                    if img.get("base64"):
                        image_vault[img_name] = img.get("base64")
                        # Update the markdown reference to the new unique image name
                        full_markdown = full_markdown.replace(img.get('name'), img_name)

            full_markdown_list.append(chunk_markdown)

            stop = False # Success
            with open(file_path.replace("data/","data/Parsed_").replace(".pdf",".md"), "a", encoding="utf-8") as f:
            # with open("data/Parsed_Kazakh_history_6_samle.md", "a", encoding="utf-8") as f:
                f.write(chunk_markdown)
            print()
            print(f"Length of parsed pages: {len(chunk_markdown)}")
            print(f"Current total markdown length: {len(full_markdown)}")
            
        except ValueError as e:
            retry_count += 1
            print(f"Chunk: {idx+1}, Attempt: {retry_count}, Error: {e}")
            time.sleep(60) # Wait before retry
        except FileNotFoundError as e:
            stop = False
        except Exception as e:
            time.sleep(1) # Wait before retry
            
    # Clean up temp file
    if os.path.exists(chunk_file):
        os.remove(chunk_file)

print("--- ALL CHUNKS PROCESSED ---")
print(f"Total markdown length: {len(full_markdown)}")
print(f"Total images extracted: {len(image_vault)}")

Total chunks to process: 120
[1/120] | data/temp_chunks/chunk_0_2.pdf
Started parsing the file under job_id 9ec8855a-1fdf-4e05-a0ed-8b1f562ced6b

Processing image: chunk_0_page_1.jpg

Processing image: chunk_0_page_2.jpg

Length of parsed pages: 1824
Current total markdown length: 1824
[2/120] | data/temp_chunks/chunk_2_4.pdf
Error while parsing the file 'data/temp_chunks/chunk_2_4.pdf': Event loop is closed
[2/120] | data/temp_chunks/chunk_2_4.pdf
Started parsing the file under job_id 12e85657-5805-486a-9a82-5327845ac677

Processing image: chunk_1_page_1.jpg

Processing image: chunk_1_page_2.jpg

Length of parsed pages: 3514
Current total markdown length: 5338
[3/120] | data/temp_chunks/chunk_4_6.pdf
Error while parsing the file 'data/temp_chunks/chunk_4_6.pdf': Event loop is closed
[3/120] | data/temp_chunks/chunk_4_6.pdf
Started parsing the file under job_id 45f99824-1e7e-4c06-894e-fe1af5b3fa5d

Processing image: chunk_2_page_1.jpg

Processing image: chunk_2_page_2.jpg

Length of pa

In [41]:
print(len(full_markdown))

480020


In [69]:
full_markdown = full_markdown.replace("```markdown", "").replace("```", "")

In [168]:

# Настраиваем разделитель по заголовкам Markdown
headers_to_split_on = [
    ("#", "Header_1"),
    ("##", "Header_2"),
    ("###", "Header_3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    strip_headers=False, # Оставляем заголовки внутри текста для векторизации
    
)

# Режем текст на логические разделы (чанки)
semantic_chunks = markdown_splitter.split_text(full_markdown)
semantic_chunks

[Document(metadata={}, page_content='START OF PAGE: 1  \nН.С. Бакина, Н.Т. Жанақова, Қ.Қ. Сулейменова'),
 Document(metadata={'Header_1': 'ҚАЗАҚСТАН ТАРИХЫ'}, page_content='# ҚАЗАҚСТАН ТАРИХЫ  \nЖалпы білім беретін мектептің 6-сыныбына арналған оқулық'),
 Document(metadata={'Header_1': '6'}, page_content='# 6  \nҚазақстан Республикасының Білім және ғылым министрлігі ұсынған\nА.Байтұрсынұлы атындағы Тіл білімі институтының сарапшыларымен келісілді  \nАлматы «Атамұра» 2018  \nEND OF PAGE: 1\nSTART OF PAGE: 2  \nӘОЖ 373.167.1\nКБЖ 63.3 (5 Қаз) я 72\nБ 21  \n*Оқулық Қазақстан Республикасының Білім және ғылым министрлігі бекіткен негізгі орта білім беру деңгейінің 5–9-сыныптарына арналған «Қазақстан тарихы» пәнінің жаңартылған мазмұндағы Типтік оқу бағдарламасына сәйкес дайындалды.*  \nПікір жазғандар:\nҚабылдинов З. Е., тарих ғылымдарының докторы, профессор\nТөлебаев Т.Ә., тарих ғылымдарының докторы, профессор\nШеглов С.Г., ҚР ағарту ісінің үздігі, жоғары санатты тарих пәнінің мұғалімі\nМәу

In [199]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

# Initialize splitter
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024,      # Target size in characters (or tokens if configured)
    chunk_overlap=100,    # Overlap to preserve context at edges
    separators=["\n\n", "\n", " "], # Priority order for splitting
    model_name="gpt-4o"
)

final_docs = token_splitter.split_documents(semantic_chunks)

token_counts_2 = []
for i, doc in enumerate(final_docs):
    token_counts_2.append(len(enc.encode(doc.page_content)))

min(token_counts_2), max(token_counts_2), np.mean(token_counts_2), np.median(token_counts_2), sum(token_counts_2), np.std(token_counts_2)

(6,
 1004,
 np.float64(525.0123456790124),
 np.float64(467.5),
 170104,
 np.float64(368.5072907072184))

In [200]:
len(final_docs), len(semantic_chunks)

(324, 220)

In [201]:
# page_t = 1
# for doc in final_docs:
#     if "START OF PAGE" in doc.page_content or "END OF PAGE" in doc.page_content:
#         start_page_numbers = re.findall(r"START OF PAGE: (\d+)", doc.page_content)
#         end_page_numbers = re.findall(r"END OF PAGE: (\d+)", doc.page_content)
#         page_t = max(start_page_numbers + end_page_numbers)
#     else:
#         doc.page_content = f"PAGE {page_t}\n\n" + doc.page_content

page_t = 1
for doc in final_docs:
    # 1. Find ALL page markers in this specific chunk
    # matches will be a list like ['5', '6']
    page_matches = re.findall(r"(?:START|END) OF PAGE: (\d+)", doc.page_content)
    
    # 2. Convert to integers and sort unique
    if page_matches:
        found_pages = sorted(list(set(map(int, page_matches))))
        # Update your tracker if needed, or just use these pages
        page_t = found_pages[-1] 
    else:
        found_pages = [page_t] # Fallback to last known page
        
    # 3. Add to metadata (Critical for citations!)
    doc.metadata["pages"] = found_pages
    
    # 4. CLEAN the text. Don't embed "START OF PAGE 5". It confuses the semantic meaning.
    # Remove the markers from the content
    doc.page_content = re.sub(r"(START|END) OF PAGE: \d+\n?", "", doc.page_content).strip()

In [202]:
print(f"Создано {len(final_docs)} логических разделов. Начинаем эмбеддинг...")


filtered_docs = [doc for doc in final_docs if doc.page_content.strip()]

# 2. Now extract text from the clean list
text_documents = [doc.page_content for doc in filtered_docs]

text_documents

Создано 324 логических разделов. Начинаем эмбеддинг...


['Н.С. Бакина, Н.Т. Жанақова, Қ.Қ. Сулейменова',
 '# ҚАЗАҚСТАН ТАРИХЫ  \nЖалпы білім беретін мектептің 6-сыныбына арналған оқулық',
 '# 6  \nҚазақстан Республикасының Білім және ғылым министрлігі ұсынған\nА.Байтұрсынұлы атындағы Тіл білімі институтының сарапшыларымен келісілді  \nАлматы «Атамұра» 2018  \n  \nӘОЖ 373.167.1\nКБЖ 63.3 (5 Қаз) я 72\nБ 21  \n*Оқулық Қазақстан Республикасының Білім және ғылым министрлігі бекіткен негізгі орта білім беру деңгейінің 5–9-сыныптарына арналған «Қазақстан тарихы» пәнінің жаңартылған мазмұндағы Типтік оқу бағдарламасына сәйкес дайындалды.*  \nПікір жазғандар:\nҚабылдинов З. Е., тарих ғылымдарының докторы, профессор\nТөлебаев Т.Ә., тарих ғылымдарының докторы, профессор\nШеглов С.Г., ҚР ағарту ісінің үздігі, жоғары санатты тарих пәнінің мұғалімі\nМәуленқұл Н.Ә., жоғары санатты тарих пәнінің мұғалімі',
 '### Шартты белгілер:  \n| ![icon1](./images/icon1.png) | Сұрақтар мен тапсырмалар |\n|------------------------------|--------------------------|\n| !

In [203]:
len(text_documents), len(final_docs), len(filtered_docs)

(324, 324, 324)

In [204]:
def sparse_documents(corpus):
    output = sparse_model.encode(corpus,
    return_dense=False, 
    return_sparse=True, 
    return_colbert_vecs=False)

    batch_keys = []
    batch_values = []
    for batch in output["lexical_weights"]:
        batch_keys.append([int(k) for k in batch.keys()])
        batch_values.append([float(v) for v in batch.values()])
    return batch_keys, batch_values
    

# Генерация векторов (Dense + Sparse)
print("Генерация векторов...")
dense_vectors = dense_model.embed_documents(text_documents)

print("Генерация разреженных векторов...")
s_indices_batch, s_values_batch = sparse_documents(text_documents)

Генерация векторов...
Генерация разреженных векторов...


Inference Embeddings: 100%|██████████| 27/27 [01:53<00:00,  4.21s/it]


In [205]:
# Note the "metadata." prefix
nested_fields = ["metadata.discipline", "metadata.publisher", "metadata.grade", "metadata.pages"]

for field in nested_fields:
    client.create_payload_index(
        collection_name=collection_name,
        field_name=field,
        field_schema=models.PayloadSchemaType.KEYWORD
    )
    print(f"Index created for nested field: '{field}'")

Index created for nested field: 'metadata.discipline'
Index created for nested field: 'metadata.publisher'
Index created for nested field: 'metadata.grade'
Index created for nested field: 'metadata.pages'


In [206]:
for idx, text_content in enumerate(final_docs):
    print(final_docs[idx].metadata.get("pages", []))

[1]
[1]
[1, 2]
[2, 3]
[3, 4]
[4, 5]
[5, 6, 7]
[7, 8]
[8, 9]
[9, 10, 11]
[11]
[11]
[11, 12]
[12, 13]
[13, 14]
[14, 15, 16]
[16]
[16, 17]
[17, 18]
[18, 19, 20]
[20]
[20, 21]
[21, 22, 23]
[23, 24]
[24]
[24]
[24, 25]
[25]
[25, 26]
[26]
[26]
[26, 27]
[27, 28]
[28]
[28, 29]
[29]
[29, 30, 31]
[30, 31, 32]
[32]
[32]
[32]
[32, 33]
[33]
[33]
[33, 34]
[34]
[34]
[34]
[34, 35]
[35, 36, 37]
[37, 38]
[38, 39]
[39]
[39]
[39, 40, 41]
[41, 42]
[42, 43]
[43, 44]
[44]
[44]
[44, 45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45]
[45, 46, 47]
[47, 48]
[48]
[48]
[48, 49]
[49]
[49, 50]
[50, 51, 52]
[52]
[52]
[52]
[52, 53, 54]
[54, 55]
[55, 56, 57]
[57]
[57]
[57]
[57]
[57, 58]
[58, 59]
[59, 60]
[60, 61]
[61]
[61]
[61, 62]
[62, 63, 64]
[63, 64, 65]
[65]
[65, 66, 67]
[67, 68]
[68, 69]
[69, 70]
[70, 71]
[71]
[71]
[71, 72, 73]
[73]
[73]
[73, 74, 75]
[75, 76]
[76]
[76]
[76, 77, 78]
[78, 79]
[79]
[79, 80]
[80, 81]
[81, 82]
[82]
[82]
[82]
[82]
[82, 8

In [207]:

point_ids = [str(uuid.uuid4()) for _ in text_documents]

# Сохраняем в Qdrant
points = [models.PointStruct(
    id=point_ids[i],
    vector={
        "voyage-dense": dense,
        "bge-sparse": models.SparseVector(indices=keys, values=vals)
    },
    payload={
        "page_content": text,
        "metadata": {
            "discipline": DISCIPLINE,
            "grade": GRADE,
            "publisher": PUBLISHER,
            "pages": filtered_docs[i].metadata.get("pages", [])
        }
    }
) for i, (text, dense, keys, vals) in enumerate(zip(text_documents, dense_vectors, s_indices_batch, s_values_batch))] 

client.upsert(collection_name=collection_name, points=points)


print("✅ Индексация по заголовкам завершена!")

✅ Индексация по заголовкам завершена!


In [178]:
def sparse_query(query):
    output = sparse_model.encode(query,
    return_dense=False, 
    return_sparse=True, 
    return_colbert_vecs=False)
    return [int(k) for k in output["lexical_weights"].keys()], [float(v) for v in output["lexical_weights"].values()]

In [179]:

# 3. Поиск (остается таким же, но теперь возвращает целые темы)
def hybrid_search(query, k=20):
    d_vec = dense_model.embed_query(query)
    s_indices, s_values = sparse_query(query)

    return client.query_points(
        collection_name=collection_name,
        prefetch=[
            # Первый поток: Dense (Gemini)
            models.Prefetch(
                query=d_vec,
                using="voyage-dense", # Имя dense вектора
                limit=k,
            ),
            # Второй поток: Sparse (BGE-M3)
            models.Prefetch(
                query=models.SparseVector(indices=s_indices, values=s_values),
                using="bge-sparse", # Имя sparse вектора
                limit=k,
            ),
        ],
        # Объединяем результаты через RRF (Reciprocal Rank Fusion)
        # Это лучший способ для гибридного поиска
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=k,
    )

docs = hybrid_search("канси деген кім?")
docs

QueryResponse(points=[ScoredPoint(id='0d8b8468-e0b5-4153-a79c-9ba7cb81e306', version=9, score=0.5, payload={'page_content': 'Орхон және Енисей өзендері алқаптарындағы ескерткіштер түркі руна жазуының ғажайып үлгісі саналады. Ескерткіштер осы өңірден табылғандықтан «_Орхон-Енисей жазуы_» деп аталды. Олардың ішінде Білге қаған мен оның інісі, қолбасшы Құлтегіннің және қаған кеңесшісі _Тоныкөктің_ құлпытастары тарихи құнды жәдігерлер саналады.  \nҚұлпытастарда түркі халқының неден сақтанып, нені ұстанатыны қашалып жазылған. Бұрынғы және қазіргі ұрпақты мақтау мен кінәлау, тыңдаушысына ығытқ да үңдеу тастап отыруы бұл мәтіндерді әсерлі етеді. Қағанның «мәңгі тасқа сызып жазған», «жыры мен сөзінің» авторы Йоллығ-тегін еді. Йоллығ-тегін – түркі тілдес әдебиеттер тарихында өз атымен аталған алғашқы автор.  \n> Түркілерде жазу қандай рөл атқарды?  \nБарлық жазулар түркі қоғамының бүкіл жұртшылығына арналған үндеулерден тұрады. Үндеу авторлары халықты қаған маңына бірігіп, топтасуға шақырады.  

In [181]:
for doc in docs.points:
    print(doc)

id='0d8b8468-e0b5-4153-a79c-9ba7cb81e306' version=9 score=0.5 payload={'page_content': 'Орхон және Енисей өзендері алқаптарындағы ескерткіштер түркі руна жазуының ғажайып үлгісі саналады. Ескерткіштер осы өңірден табылғандықтан «_Орхон-Енисей жазуы_» деп аталды. Олардың ішінде Білге қаған мен оның інісі, қолбасшы Құлтегіннің және қаған кеңесшісі _Тоныкөктің_ құлпытастары тарихи құнды жәдігерлер саналады.  \nҚұлпытастарда түркі халқының неден сақтанып, нені ұстанатыны қашалып жазылған. Бұрынғы және қазіргі ұрпақты мақтау мен кінәлау, тыңдаушысына ығытқ да үңдеу тастап отыруы бұл мәтіндерді әсерлі етеді. Қағанның «мәңгі тасқа сызып жазған», «жыры мен сөзінің» авторы Йоллығ-тегін еді. Йоллығ-тегін – түркі тілдес әдебиеттер тарихында өз атымен аталған алғашқы автор.  \n> Түркілерде жазу қандай рөл атқарды?  \nБарлық жазулар түркі қоғамының бүкіл жұртшылығына арналған үндеулерден тұрады. Үндеу авторлары халықты қаған маңына бірігіп, топтасуға шақырады.  \n![Көне түркі тас жазуының үзіндісі]

In [208]:
from qdrant_client import models

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

import voyageai
from langchain_voyageai import VoyageAIRerank


# Initialize Voyage client
vo = voyageai.Client(api_key=VOYAGE_API)

# --- 1. Setup Models ---
# LLM (Gemini 1.5 Flash for speed/cost)
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=1, api_key=GEMINI_API_KEY)

# --- 2. Custom Hybrid Retrieval Logic ---

def hybrid_retriever_func(query: str, metadata_filter: dict = None):
    """
    Custom function to perform Hybrid Search in Qdrant 
    and return content + images from the payload.
    """

    qdrant_filter = None
    if metadata_filter:
        conditions = []
        for key, value in metadata_filter.items():
            # Мы используем "metadata.key", так как в payload мы вложили словарь metadata
            conditions.append(
                models.FieldCondition(
                    key=f"metadata.{key}", 
                    match=models.MatchValue(value=value)
                )
            )
        qdrant_filter = models.Filter(must=conditions)

    # Generate Dense Vector
    query_dense = dense_model.embed_query(query)
    
    # Generate Sparse Vector
    keys, vals = sparse_query(query)
    query_sparse = models.SparseVector(
        indices=keys,
        values=vals
    )

    # Perform Hybrid Search using RRF (Reciprocal Rank Fusion)
    search_results = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(query=query_dense, using="voyage-dense", limit=20, filter=qdrant_filter),
            models.Prefetch(query=query_sparse, using="bge-sparse", limit=20, filter=qdrant_filter),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=40,
        with_payload=True
    )
    if not search_results.points:
        return {"context_text": "Информация не найдена.", "images": []}
    

    # --- STEP 2: VOYAGE RERANK 2.5 (Precision) ---
    # Prepare candidates for the Reranker
    candidate_texts = [hit.payload['page_content'] for hit in search_results.points]

    
    # rerank-2 is the model name for Voyage's latest reranker
    rerank_results = vo.rerank(
        query=query, 
        documents=candidate_texts, 
        model="rerank-2.5", 
        top_k=5 # We only want the absolute best 5 now
    )

    # --- STEP 3: FORMAT RESULTS ---
    reranked_docs = []


    for r in rerank_results.results:
        # Get the original index from the candidates
        idx = r.index
        hit = search_results.points[idx]
        
        # Add text and the images associated with this specific chunk
        reranked_docs.append(f"""
--- Context Segment (Relevance Score: {r.relevance_score:.4f}) ---
Кітап атауы: {hit.payload.get('metadata', {}).get('discipline', 'Unknown')}
Сынып: {hit.payload.get('metadata', {}).get('grade', 'Unknown')}
Баспа: {hit.payload.get('metadata', {}).get('publisher', 'Unknown')}
Беттер: {', '.join(map(str, hit.payload.get('metadata', {}).get('pages', [])))}

{hit.payload['page_content']}""")
        
    

    return {"context_text": "\n\n".join(reranked_docs)}

# --- 3. Construct the Prompt ---

prompt_template = """
Сен Қазақстан тарихынан ҰБТ-ға (Ұлттық бірыңғай тестілеу) дайындайтын кәсіби репетиторсың. 
Сенің міндетің - тек қана берілген контекст негізінде студентке жауап беру.

Нұсқаулықтар:
1. Жауапты нақты фактілермен (жылдар, есімдер, оқиғалар) негізде.
2. Егер контекстте ақпарат болмаса, "Мәтінде бұл сұраққа жауап жоқ" деп айт.
3. Жауаптың соңында міндетті түрде пайдаланылған дереккөздерді көрсет. (Кітап атауы, Сыныбы, Баспасы, Кытап беттерінің нөмірлері)

Контекст:
{context}

Сұрақ: {question}
"""

def build_multimodal_prompt(input_dict):
    """
    Constructs the final list of messages (text + images) for Gemini.
    """
    question = input_dict["question"]
    context_data = input_dict["context_data"]
    
    # Text Part
    prompt_text = prompt_template.format(
        context=context_data["context_text"], 
        question=question
    )
    
    message_content = [{"type": "text", "text": prompt_text}]
    
    # # Image Part (Base64)
    # for img_b64 in context_data["images"][:3]: # Limit to 3 images for token safety
    #     message_content.append({
    #         "type": "image_url",
    #         "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}
    #     })
        
    return [HumanMessage(content=message_content)]

# --- 4. The LangChain Expression Language (LCEL) Chain ---

from langchain_core.messages import HumanMessage

rag_chain_w_sources = {
        "context_data": lambda x: hybrid_retriever_func(x["question"], x.get("filter")),
        "question": lambda x: x["question"]
    }| RunnablePassthrough().assign(
    response=(
    RunnableLambda(build_multimodal_prompt)
    | llm
    | StrOutputParser()
    )
)


In [209]:
query_params = {
    "question": "Әмір темір мамайды жеңдіма?",
    "filter": {
        "grade": "6 сынып",
        "discipline": "Қазақстан тарихы"
    }
}

response = rag_chain_w_sources.invoke(query_params)
response

{'context_data': {'context_text': '\n--- Context Segment (Relevance Score: 0.6953) ---\nКітап атауы: Қазақстан тарихы\nСынып: 6 сынып\nБаспа: Атамұра\nБеттер: 123, 124, 125\n\n# XIII ҒАСЫРДАҒЫ МОНГОЛ ЖАУЛАП АЛУ ЖОРЫҚТАРЫ  \n![Map](image)  \n- Монғол тайпаларының 1206 жылы Шыңғысханның қол астына біріккендегі орналасқан жері\n- 1227 жылы қайтыс болғанға дейінгі Шыңғысханның жорықтары\n- XIII ғасырдағы Шыңғысхан ұрпақтарының жорықтап алған жерлері\n- Монғолдардың қол астындағы аумақтар\n- Монғолдарға алым-салық төлеуін аумақтар\n- Монғол қолынан қалтылған қаласы ірі мемлекеттер\n- Васты шайқас өткен жерлер мен лагаптар  \n123  \n  \n1380 жылы Алтын Орда билеушісі Мамайды Куликов шайқасында Мәскеу князы Дмитрий Донский әскері талқандады. Шыңғысхан ұрпағы Тоқтамыс хан Мамайдың жеңілісін пайдаланып, Әмір Темірдің қолдауымен Алтын Ордадағы билікті тартып алды. Әскери жеңістермен өз билігін нығайтуға ұмтылған ол **1382 жылы** Мәскеуді өртеп, орыс князьдарының салық төлеуін қалпына келтірді.  

In [210]:
print(response["response"])

Берілген мәтін негізінде сұрағыңызға жауап:

**Жоқ, Әмір Темір Мамайды жеңген жоқ.**

Мәтіндегі деректерге сүйенсек:
1.  **1380 жылы** Алтын Орда билеушісі Мамайды **Куликов шайқасында** Мәскеу князы **Дмитрий Донскийдің** әскері талқандаған.
2.  Шыңғысхан ұрпағы **Тоқтамыс хан** Мамайдың осы жеңілісін пайдаланып, Әмір Темірдің қолдауымен Алтын Ордадағы билікті тартып алған.
3.  Әмір Темір Мамаймен емес, кейіннен күшейіп кеткен **Тоқтамыс ханмен** соғысқан. Ол **1389, 1391** және **1395 жылдардағы** үш жорығы нәтижесінде Тоқтамыстың әскерін талқандап, Алтын Орданы құлатқан.

**Дереккөз:**
*   Кітап атауы: Қазақстан тарихы
*   Сыныбы: 6 сынып
*   Баспасы: Атамұра
*   Беттері: 123, 135-136


In [188]:
for key, val in response["context_data"].items():
    print(f"{val}")


--- Context Segment (Relevance Score: 0.6836) ---
Кітап атауы: Қазақстан тарихы
Сынып: 6 сынып
Баспа: Атамұра
Беттер: 123, 124

# XIII ҒАСЫРДАҒЫ МОНГОЛ ЖАУЛАП АЛУ ЖОРЫҚТАРЫ  
![Map](image)  
- Монғол тайпаларының 1206 жылы Шыңғысханның қол астына біріккендегі орналасқан жері
- 1227 жылы қайтыс болғанға дейінгі Шыңғысханның жорықтары
- XIII ғасырдағы Шыңғысхан ұрпақтарының жорықтап алған жерлері
- Монғолдардың қол астындағы аумақтар
- Монғолдарға алым-салық төлеуін аумақтар
- Монғол қолынан қалтылған қаласы ірі мемлекеттер
- Васты шайқас өткен жерлер мен лагаптар  
123  
  
1380 жылы Алтын Орда билеушісі Мамайды Куликов шайқасында Мәскеу князы Дмитрий Донский әскері талқандады. Шыңғысхан ұрпағы Тоқтамыс хан Мамайдың жеңілісін пайдаланып, Әмір Темірдің қолдауымен Алтын Ордадағы билікті тартып алды. Әскери жеңістермен өз билігін нығайтуға ұмтылған ол **1382 жылы** Мәскеуді өртеп, орыс князьдарының салық төлеуін қалпына келтірді.  
Көп уақыт өтпей Тоқтамыс Жошы ұрпақтарының барлық иелікте